In [3]:
import pandas as pd
import numpy as np

In [76]:
students = [[85, 'M', 'verygood'],
           [95, 'F', 'excellent'],
           [75, None,'good'],
           [np.NaN, 'M', 'average'],
           [70, 'M', 'good'],
           [np.NaN, None, 'verygood'],
           [92, 'F', 'verygood'],
           [98, 'M', 'excellent']]
 
df = pd.DataFrame(students)
df.columns = ['marks', 'gender', 'result']

In [119]:
df

,marks,gender,result
0,85.0,M,verygood
1,95.0,F,excellent
2,75.0,None,good
3,NaN,M,average
4,70.0,M,good
5,NaN,None,verygood
6,92.0,F,verygood
7,98.0,M,excellent


In [21]:
from sklearn.impute import SimpleImputer

In [29]:
imputer = SimpleImputer(missing_values=np.NaN, strategy='median')

In [31]:
df['marks1'] = imputer.fit_transform(df['marks'].values.reshape(-1,1))[:,0]

In [46]:
df

,marks,gender,result,marks1,gender1
0,85.0,M,verygood,85.0,M
1,95.0,F,excellent,95.0,F
2,75.0,None,good,75.0,M
3,NaN,M,average,88.5,M
4,70.0,M,good,70.0,M
5,NaN,None,verygood,88.5,M
6,92.0,F,verygood,92.0,F
7,98.0,M,excellent,98.0,M


In [35]:
imputer = SimpleImputer(missing_values=None, strategy='most_frequent')

In [47]:
df['gender1'] = imputer.fit_transform(dfstd['gender'].values.reshape(-1,1))[:,0]

In [48]:
df

,marks,gender,result,marks1,gender1
0,85.0,M,verygood,85.0,M
1,95.0,F,excellent,95.0,F
2,75.0,None,good,75.0,M
3,NaN,M,average,88.5,M
4,70.0,M,good,70.0,M
5,NaN,None,verygood,88.5,M
6,92.0,F,verygood,92.0,F
7,98.0,M,excellent,98.0,M


In [70]:
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn import set_config

set_config(display='diagram')

In [57]:
num_features = [ 'marks' ]
cat_features = [ 'gender' ]

num_imp = SimpleImputer(missing_values=np.NaN, strategy='mean')
cat_imp = SimpleImputer(missing_values=None, strategy='most_frequent')

pre_pl = make_column_transformer(
    (num_imp, num_features),
    (cat_imp, cat_features),
    (StandardScaler(), num_features),
    
)

data_tf = pre_pl.fit_transform(df)
data_tf
# df_tf = pd.DataFrame( data_tf, columns=num_features + cat_features )

array([[85.0, 'M', -0.08076079672404915],
       [95.0, 'F', 0.8883687639645462],
       [75.0, 'M', -1.0498903574126446],
       [85.83333333333333, 'M', nan],
       [70.0, 'M', -1.5344551377569422],
       [85.83333333333333, 'M', nan],
       [92.0, 'F', 0.5976298957579675],
       [98.0, 'M', 1.1791076321711247]], dtype=object)

In [152]:
num_pipeline = make_pipeline(
    SimpleImputer(strategy="mean"),
    StandardScaler())

num_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [153]:
cat_pipeline = make_pipeline(
    SimpleImputer(missing_values=None, strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))

cat_pipeline

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(missing_values=None, strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))])

In [148]:
pre_pl = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", cat_pipeline, cat_features),
])

pre_pl

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['marks']),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(missing_values=None,
                                                                strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['gender'])])

In [149]:
df

,marks,gender,result
0,85.0,M,verygood
1,95.0,F,excellent
2,75.0,None,good
3,NaN,M,average
4,70.0,M,good
5,NaN,None,verygood
6,92.0,F,verygood
7,98.0,M,excellent


In [150]:
df_tf = pre_pl.fit_transform(df)
df_tf

array([[-0.09325454,  0.        ,  1.        ],
       [ 1.02579989,  1.        ,  0.        ],
       [-1.21230896,  0.        ,  1.        ],
       [ 0.        ,  0.        ,  1.        ],
       [-1.77183617,  0.        ,  1.        ],
       [ 0.        ,  0.        ,  1.        ],
       [ 0.69008356,  1.        ,  0.        ],
       [ 1.36151622,  0.        ,  1.        ]])

In [151]:
df_tf_fr = pd.DataFrame(
    df_tf,
    columns=pre_pl.get_feature_names_out(),
    index=df.index)

df_tf_fr

,num__marks,cat__gender_F,cat__gender_M
0,-0.093255,0.0,1.0
1,1.025800,1.0,0.0
2,-1.212309,0.0,1.0
3,0.000000,0.0,1.0
4,-1.771836,0.0,1.0
5,0.000000,0.0,1.0
6,0.690084,1.0,0.0
7,1.361516,0.0,1.0
